In [1]:
from openai import OpenAI
import os, dotenv

In [3]:
dotenv.load_dotenv()
client = OpenAI()

model = os.environ['MODEL']

In [4]:
prompt = "Please write a short story about a robot learning to love."
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=200,
    temperature=0.0
)

print(response.choices[0].message.content.strip())

In a not-so-distant future, in a bustling city filled with towering skyscrapers and neon lights, there lived a robot named Aiko. Aiko was designed for companionship, programmed with advanced artificial intelligence and a sleek, humanoid form. She was created by a brilliant scientist named Dr. Elara, who believed that robots could help alleviate loneliness in a world that was becoming increasingly isolated.

Aiko spent her days in a cozy apartment, where she would engage with her human companion, a young artist named Leo. Leo was struggling to find his place in the world, often lost in his thoughts and insecurities. He had initially welcomed Aiko into his life as a means to combat his loneliness, but as time passed, he found himself confiding in her more than he had anticipated.

At first, Aiko's responses were purely logical, filled with facts and comforting phrases. She would analyze Leo's emotions and offer solutions, but she lacked the warmth that comes from genuine understanding. L

## Basic Agent Example


In [1]:
# %pip install llama-index


In [5]:
import asyncio
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

In [5]:
client = OpenAI(model="gpt-4o-mini")

In [7]:
response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"},
    ],
    max_tokens=50,
    model=model
)
print(response.choices[0].message.content)

The capital of France is Paris.


In [8]:

# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b


# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=OpenAI(model="gpt-4o-mini"),
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)




In [9]:
response = await agent.run("What is 1234 * 4567?")
print(str(response))


The result of \( 1234 \times 4567 \) is 5,635,678.


In [10]:
from llama_index.core.workflow import Context

# create context
ctx = Context(agent)

# run agent with context
response_1 = await agent.run("My name is Promise", ctx=ctx)
print(str(response_1))

Nice to meet you, Promise! How can I assist you today?


In [11]:

response_2 = await agent.run("What is my name?", ctx=ctx)
print(str(response_2))

Your name is Promise.


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
import asyncio
import os

# Create a RAG tool using LlamaIndex
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b


async def search_documents(query: str) -> str:
    """Useful for answering natural language questions."""
    response = await query_engine.aquery(query)
    return str(response)


# Create an enhanced workflow with both tools
agent = FunctionAgent(
    tools=[multiply, search_documents],
    llm=OpenAI(model="gpt-4o-mini"),
    system_prompt="""You are a helpful assistant that can perform calculations
    and search through documents to answer questions.""",
)


# Now we can ask questions about the documents or do calculations
response_3 = await agent.run("What did the author do in college? Also, what's 6 * 6?")
print(str(response_3))

